In [3]:
%%time
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
from snowflake.ml.registry import Registry
import pandas as pd

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
CPU times: user 1.59 s, sys: 903 ms, total: 2.49 s
Wall time: 9.69 s


In [4]:
import requests
exp_id="aaaawwwwweeeer"
project_id="fd0cf79c-8118-43a7-8f0b-059e8f78227a"
executions_url = f"http://monitor-backend-service/monitor/api/v4/executions/search?objectId={exp_id}&projectId={project_id}&jobType=EXPERIMENT&profile=DETAILED"

temp_head = {
    "X-Auth-Userid": "aditya1.singh2@fosfor.com",
    "X-Auth-Username": "aditya1.singh2@fosfor.com",
    "X-Auth-Email": "aditya1.singh2@fosfor.com",
    "X-Project-Id": "fd0cf79c-8118-43a7-8f0b-059e8f78227a"
}

In [5]:
response = requests.get(executions_url, headers=temp_head)
print(response.text)

{"totalRecordCount":32,"data":[{"jobinstanceId":109574,"objectId":"aaaawwwwweeeer","projectId":"fd0cf79c-8118-43a7-8f0b-059e8f78227a","projectName":null,"jobObjectName":"exp_001","jobScheduleId":null,"jobInstanceInputParameter":"{}","jobType":"EXPERIMENT","jobStatus":"DIDNOTRUN","finalStatus":"DIDNOTRUN","jobCreatedby":"aditya1.singh2@fosfor.com","jobCreatedts":"2024-07-09T09:07:50.960+00:00","jobUpdatedby":"aditya1.singh2@fosfor.com","jobUpdatedts":"2024-07-09T09:07:50.960+00:00","applicationId":"","executionId":null,"runID":"109574","versionNumber":null,"runEngine":"SNOWFLAKE","attemptId":109678,"workflowId":null,"jobConfig":"{\"snowflake_query\": CALL experiment_stored_proc('{\"monitor_run_id\": \"MONITORRUNID\", \"experiment_id\": \"aaaawwwwweeeer\", \"experiment_name\": \"exp_001\", \"stored_procedure\": \"run_experiment\", \"algorithm_category\": \"Snowparkml\", \"algorithm_type\": \"Classification\", \"dataset_name\": \"diamonds_dataset\", \"target_column\": \"quality\", \"algor

In [4]:
%%time
monitor_resp = [
    {
        "run_id": "RUN_0001",
        "algorithm_used": "XGBClassifier",
        "status": "Successful"
    },
    {
        "run_id": "RUN_0001_11",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful"
    },
    {
        "run_id": "ef47a0a5",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running"
    },
    {
        "run_id": "26a15f9c",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running"
    }
]

# monitor_resp = []
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","fd0cf79c-8118-43a7-8f0b-059e8f78227a") #,"")0e0fb803-22db-4d88-9f2f-f6f75b6abcf0
query ="SELECT METADATA:metrics as model_metrics, COMMENT as run_id FROM INFORMATION_SCHEMA.MODEL_VERSIONS where MODEL_NAME like '%EXP_001%';"
registry_resp = session.sql(query).to_pandas()
registry_resp.rename(columns={'RUN_ID': 'run_id', 
                              'MODEL_METRICS': 'model_metrics'}, inplace=True)
monitor_resp = pd.DataFrame(monitor_resp)
final_resp = pd.merge(monitor_resp, registry_resp, on='run_id', how='left')
print({'data': final_resp.to_dict(orient='records')})

{'data': [{'run_id': 'RUN_0001', 'algorithm_used': 'XGBClassifier', 'status': 'Successful', 'model_metrics': '{\n  "algo_details": {\n    "snowflake.ml.modeling.xgboost.XGBClassifier": null\n  },\n  "algorithm_type": "classification",\n  "dataset": "AIRLINE_DEP_DELAY_10K",\n  "exp_name": "Final_recipe",\n  "id": "7bbb5061-54d4-4862-8d47-7fbee388a4d1",\n  "model_metrics": {\n    "accuracy_score": 0.803922,\n    "f1_score": 0.23076923076923075,\n    "log_loss": 7.0706121046545105,\n    "precision_score": 0.4878048780487805,\n    "recall_score": 0.15113350125944586,\n    "roc_auc": 0.5563945047380613\n  },\n  "project_id": "0e0fb803-22db-4d88-9f2f-f6f75b6abcf0",\n  "run_id": "6d0370b7-7aab-4394-884a-266bc284d07a",\n  "source": "EXP",\n  "target_column": "DEP_DEL15"\n}'}, {'run_id': 'RUN_0001_11', 'algorithm_used': 'DecisionTreeClassifier', 'status': 'Successful', 'model_metrics': '{\n  "algo_details": {\n    "snowflake.ml.modeling.xgboost.XGBClassifier": null\n  },\n  "algorithm_type": "c

In [8]:
monitor_resp=[]
if monitor_resp:
    print("TRUE")
else:
    print("NOT TRUE")

NOT TRUE


In [3]:
%%time
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running",
        "model_metrics": {}
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running",
        "model_metrics": {}
    }
]

def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","fd0cf79c-8118-43a7-8f0b-059e8f78227a")
reg = Registry(session=session)

model_list = session.sql("show models like '%exp6789%'").collect()
for model in model_list:
    model_name = model["name"]
    model_run_id = model_name[-36:].replace('_','-').lower()
    model_obj = reg.get_model(model_name)
    metrics = {
        "model_metrics" : model_obj.default.get_metric('model_metrics')
    }
    for run in monitor_resp:
        if run.get('run_id') == model_run_id:
            run.update(metrics)
print(monitor_resp)

[{'run_id': '8042932c-101a-4ddc-a808-00c092797960', 'algorithm_used': 'XGBClassifier', 'status': 'Successful', 'model_metrics': {'accuracy_score': 0.912834, 'f1_score': 0.863711823763472, 'recall_score': 0.8009236633760141, 'precision_score': 0.9371819065591582, 'roc_auc': 0.8863324874130605, 'log_loss': 3.1437201566897612}}, {'run_id': '16e3f179-bb70-4632-9a39-9a6fac4a5581', 'algorithm_used': 'DecisionTreeClassifier', 'status': 'Successful', 'model_metrics': {}}, {'run_id': 'ef47a0a5-1a74-4a14-b0fd-8dbbd322057f', 'algorithm_used': 'GradientBoostingClassifier', 'status': 'Running', 'model_metrics': {}}, {'run_id': '26a15f9c-29fd-4d55-82fb-3d38cc18e4b8', 'algorithm_used': 'RandomForestClassifier', 'status': 'Running', 'model_metrics': {}}]
CPU times: user 1.61 s, sys: 195 ms, total: 1.81 s
Wall time: 10.2 s


In [7]:
%%time
####Approach 2####
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running",
        "model_metrics": {}
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running",
        "model_metrics": {}
    }
]

def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","fd0cf79c-8118-43a7-8f0b-059e8f78227a")
reg = Registry(session=session)
resp_df = pd.DataFrame(monitor_resp)
model_list = session.sql("show models like '%exp6789%'").collect()
for model in model_list:
    model_name = model["name"]
    model_run_id = model_name[-36:].replace('_','-').lower()
    model_obj = reg.get_model(model_name)
    metrics = {
        "model_metrics" : model_obj.default.get_metric('model_metrics')
    }
    select_row = resp_df.loc[resp_df['run_id']==model_run_id]
    resp_df.loc[resp_df['run_id']==model_run_id, 'model_metrics'] = str(metrics.get('model_metrics'))   
print(resp_df)

                                 run_id              algorithm_used  \
0  8042932c-101a-4ddc-a808-00c092797960               XGBClassifier   
1  16e3f179-bb70-4632-9a39-9a6fac4a5581      DecisionTreeClassifier   
2  ef47a0a5-1a74-4a14-b0fd-8dbbd322057f  GradientBoostingClassifier   
3  26a15f9c-29fd-4d55-82fb-3d38cc18e4b8      RandomForestClassifier   

       status                                      model_metrics  
0  Successful  {'accuracy_score': 0.912834, 'f1_score': 0.863...  
1  Successful                                                 {}  
2     Running                                                 {}  
3     Running                                                 {}  
CPU times: user 922 ms, sys: 19.8 ms, total: 942 ms
Wall time: 4.72 s


In [2]:
import requests, json

In [9]:
monitor_url = "http://monitor-backend-service/monitor/api/v5/executions/search?objectId=f40619bc-16a1-4d6b-8f74-6b95c1d7c76a_bWFpbg==_U25vd3BhcmtSbkQvQnVpbGRfRGF0YXNldC5pcHluYg==&projectId=0e0fb803-22db-4d88-9f2f-f6f75b6abcf0&jobType=NOTEBOOK"
temp_head = {
    "X-Auth-Userid": "aditya1.singh2@fosfor.com",
    "X-Auth-Username": "aditya1.singh2@fosfor.com",
    "X-Auth-Email": "aditya1.singh2@fosfor.com",
    "X-Project-Id": "0e0fb803-22db-4d88-9f2f-f6f75b6abcf0"
}

In [10]:
res = requests.get(monitor_url, headers=temp_head)
res = json.loads(res.text)

In [11]:
res

{'totalRecordCount': 1,
 'data': [{'jobinstanceId': 502283,
   'objectId': 'f40619bc-16a1-4d6b-8f74-6b95c1d7c76a_bWFpbg==_U25vd3BhcmtSbkQvQnVpbGRfRGF0YXNldC5pcHluYg==',
   'projectId': '0e0fb803-22db-4d88-9f2f-f6f75b6abcf0',
   'projectName': 'SnowParkRnd',
   'jobObjectName': 'Build_Dataset.ipynb',
   'jobScheduleId': None,
   'jobInstanceInputParameter': '{"namespace":"insight"}',
   'jobType': 'NOTEBOOK',
   'jobStatus': 'FAILED',
   'finalStatus': 'FAILED',
   'jobCreatedby': 'aditya1.singh2@fosfor.com',
   'jobCreatedts': '2024-07-08T10:47:33.847+00:00',
   'jobUpdatedby': 'aditya1.singh2@fosfor.com',
   'jobUpdatedts': '2024-07-08T10:48:51.959+00:00',
   'applicationId': None,
   'executionId': None,
   'runID': '502283',
   'versionNumber': None,
   'runEngine': None,
   'attemptId': 0,
   'workflowId': None,
   'jobConfig': None}],
 'jobTypesList': None,
 'executedByList': None,
 'projectList': None}

In [12]:
class ResponseFormats:
    LIST_RUNS = {
            "run_id": "",
            "algorithm_used": "",
            "status": "",
            "model_metrics": {}
            }

In [13]:
_res_format = ResponseFormats.LIST_RUNS

In [14]:
_res_format['status']

''

In [15]:
final_resp = []
for data in res.get('data'):
    _res_format.update({"run_id": data.get('runID'), "status": data.get('jobStatus')})
    res_data = _res_format
    final_resp.append(res_data)

print(final_resp)

[{'run_id': '502283', 'algorithm_used': '', 'status': 'FAILED', 'model_metrics': {}}]


In [3]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","fd0cf79c-8118-43a7-8f0b-059e8f78227a")

In [12]:
reg = Registry(session=session)

In [13]:
model_name = 'FD0CF79C_8118_43A7_8F0B_059E8F78227A_EXP_001_RUN_0001'

In [22]:
model_obj = reg.get_model(model_name)
metrics = model_obj.default.show_metrics()


In [25]:
metrics

{'model_metrics': {'accuracy_score': 0.92473,
  'f1_score': 0.8841251148525358,
  'recall_score': 0.8361769313619897,
  'precision_score': 0.9379067021773153,
  'roc_auc': 0.9036114375207266,
  'log_loss': 2.7149916732535027},
 'project_id': 'test12345',
 'experiment_id': 'exp6789',
 'monitor_run_id': '16e3f179-bb70-4632-9a39-9a6fac4a5581',
 'algorithm_type': 'Classification',
 'algorithm': 'DecisionTreeClassifier',
 'dataset': 'EMPLOYEE_10L',
 'target_column': 'LEAVEORNOT',
 'experiment_name': 'My_run_listing_exp',
 'source': 'EXP'}

In [17]:
dataset = model_obj.default.get_metric('dataset')

In [18]:
dataset

'EMPLOYEE_10L'

In [4]:
! pip freeze | grep -i snowf

snowflake==0.10.0
snowflake-connector-python==3.11.0
snowflake-core==0.10.0
snowflake-legacy==0.10.0
snowflake-ml-python==1.5.3
snowflake-snowpark-python==1.19.0
